# Evitando UDFs en Apache Spark

Uno de los componentes que más me sorprendió en Apache Spark es que permitiera extender el vocabulario de SQL fuera de los límites de DSL, incluso, incrustando funciones de negocio escritas en diferente lenguaje Scala, Java o Python, son admitidos. Antes de evitarlos deberíamos saber ¿que son los UDFs?, ¿cómo ayudan extendiendo las capacidades de SQL?, consideraciones respecto a el rendimiento, por que debería evitar esta maravillosa funcionalidad, ¡bienvenido Column Functions!

## Contenido:
* [Prerequisitos](#head1)
* [Pero Que son los UDF en Apache Spark?](#head2)
* [Rendimiento](#head3)
* [Bienvenido: Column Functions!](#head4)
* [Replacing for Column function](#head1)
* [Conclusiones](#head1)

## Prerequisitos<a class="anchor" id="head1"></a>

Unicamente necesitaremos las siguientes importaciones en nuestro notebook:

In [60]:
import org.apache.spark.sql.functions.{col, udf}
import org.apache.spark.sql.Column

import org.apache.spark.sql.functions.{col, udf}
import org.apache.spark.sql.Column


## Pero, Que son los UDF en Apache Spark?<a class="anchor" id="head2"></a>

User-defined functions o UDF es la forma facil de crear funciones que trabajan registro a registro y que extienden la funcionalidad de SQL, permitiendo construir complejas logicas de negocio y utilizarlas como si fueran funciones nativas de SQL. Este primer ejemplo es el calculo del cuadrado en UDF, escribimos la función <i style="color:blue;">square</i>, registramos ante Spark y estará lista para su uso como función nativa de SQL. Spark serializara las funciones y las enviará a todos los procesos ejecutores en los worker para su ejecución.

In [52]:
def square(s: Long) = s * s
val squareUDF = udf(square(_:Long):Long)
spark.range(1, 10).select(squareUDF(col("id"))).show

+-------+
|UDF(id)|
+-------+
|      1|
|      4|
|      9|
|     16|
|     25|
|     36|
|     49|
|     64|
|     81|
+-------+



square: (s: Long)Long
squareUDF: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,LongType,Some(List(LongType)))


Su versión Inline más compacta con Spark SQL

In [54]:
spark.udf.register("squareUDFInline", (s: Long) => s * s)
spark.range(1, 10).createTempView("square")
spark.sql("SELECT squareUDFInline(id) from square").show

+-----------------------+
|UDF:squareUDFInline(id)|
+-----------------------+
|                      1|
|                      4|
|                      9|
|                     16|
|                     25|
|                     36|
|                     49|
|                     64|
|                     81|
+-----------------------+



## Consideraciones de rendimiento y orden de evaluación<a class="anchor" id="head3"></a>

Existe una diferencia clave asociada con el lenguaje con el cual se escribio la UDF: si es Java o Scala,  correrán sobre las JVM en las maquinas esclavas, sin embargo si la función fué escrita en Python, Spark iniciara el proceso de Python en el worker, serializara la data a un formato aceptado por Python, ejecutara la función registro a registro en el proceso de Python y finalmente retornará los resultados a la JVM.

Estas diferencias en la forma de ejecución traen implicaciones a nivel de rendimiento(<a href="https://medium.com/@QuantumBlack/spark-udf-deep-insights-in-performance-f0a95a4d8c62">Spark UDF — Deep insights in performance</a>) ofreciendo evidencia de mejores resultados a las UDFs escritas nativamente en Scala.

<img class="nh sg s t u he ai hn" srcset="https://miro.medium.com/max/552/1*FFi8Yk6mwSc6AvI-avWcYw.png 276w, https://miro.medium.com/max/1104/1*FFi8Yk6mwSc6AvI-avWcYw.png 552w, https://miro.medium.com/max/1280/1*FFi8Yk6mwSc6AvI-avWcYw.png 640w, https://miro.medium.com/max/1400/1*FFi8Yk6mwSc6AvI-avWcYw.png 700w" sizes="700px" role="presentation" src="https://miro.medium.com/max/1800/1*FFi8Yk6mwSc6AvI-avWcYw.png" width="1200" height="250">

Por que deberia evitar esta maravillosa funcionalidad aún siendo escrita en Scala? Los UDFs no son optimizados por el optimizador de consultas Catalys (<a href="https://blog.cloudera.com/working-with-udfs-in-apache-spark/">Working with UDFs in Apache Spark</a>) y las funciones nativas de SQL a menudo tendran un mejor rendimiento y deberían ser el primer enfoque considerado siempre que se pueda evitar la introducción de un UDF. Ya que no deberian ser nuestra primera elección las UDFs en Apache Spark, como podemos reemplazarlas?

## Bienvenido: Column Functions!<a class="anchor" id="head4"></a>

En su mayoria nos permiten reescribir código UDF a una función que recibe como parametro una(s) columna(s) y es capaz de retornar una(s) columna(s), se encuentran en el namespace <i style="color:blue;">org.apache.spark.sql.functions</i> (<a href="https://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/functions.html">Java</a> <a href="https://spark.apache.org/docs/2.4.5/api/scala/index.html#org.apache.spark.sql.functions">Scala</a>). Reescribamos nuestra función <i style="color:blue;">square</i>:

In [66]:
def squareFunction(col:Column):Column = col * col
spark.range(1, 10).select(squareFunction(col("id"))).show

+---------+
|(id * id)|
+---------+
|        1|
|        4|
|        9|
|       16|
|       25|
|       36|
|       49|
|       64|
|       81|
+---------+



squareFunction: (col: org.apache.spark.sql.Column)org.apache.spark.sql.Column
